# Forecast comparison among methods

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import os
from dataclasses import dataclass

@dataclass
class Methods:
    name: str
    df: pd.DataFrame

In [19]:
metrics = ['POD', 'FAR', 'CSI']
num_of_metrics = len(metrics)

def plot_forecast_comparison(data: str, frame_lead: int, save: bool = False, show_plot: bool = True):
    dir_path = f'output/{data}/{frame_lead}_frame_leads'
    file_names = os.listdir(dir_path)

    methods: list[Methods] = []

    for file_name in file_names:
        if file_name.endswith('.csv'):
            df = pd.read_csv(os.path.join(dir_path, file_name))
            methods.append(Methods(file_name[:-4], df))

    avg_time_lead = methods[0].df['leading_time (min)'].mean()

    plt.figure(figsize=(10, 4 * len(metrics)))
    indexes = range(len(methods[0].df))

    for i, metric in enumerate(metrics):
        plt.subplot(num_of_metrics, 1, i+1)
        for method in methods:
            plt.plot(indexes, method.df[metric], label=f'{method.name}: {method.df[metric].mean():.4f}')
        
        plt.title(f'{metric} Over Time ({avg_time_lead:.2f}-minutes Leads)')
        plt.xlabel('Index')
        plt.ylabel(metric)
        plt.ylim(0, 1)
        plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5)
        plt.legend(title=f'Average {metric}')

    plt.tight_layout()
    if save:
        output_path = f'output/forecast_comparison/images/{data}_{frame_lead}_frame_leads.png'
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        plt.savefig(output_path)
        print(f"Plot saved to {output_path}")
    if show_plot:
        plt.show()
    else:
        plt.close()

In [22]:
data = 'philippines_13.269_123.832'

frame_lead = 3

plot_forecast_comparison(data, frame_lead, save=True, show_plot=False)

Plot saved to output/forecast_comparison/images/philippines_13.269_123.832_3_frame_leads.png


### Export results to dataframe

In [23]:
data_list = [
    'philippines_13.269_123.832', 'KGRR', 'KARX', 'KDVN'
]
time_leads = [3, 5]
method_lists = [file_name.split('.')[0] for file_name in os.listdir('output/' + data_list[0] + '/3_frame_leads')]
metrics = ['POD', 'FAR', 'CSI']

method_dict = {method: {metric: [] for metric in metrics} for method in method_lists}
method_dict['data'] = []

for data_name in data_list:
    for lead_time in time_leads:
        # plot and explot data
        plot_forecast_comparison(data_name, lead_time, save=True, show_plot=False)

        dir_path = f'output/{data_name}/{lead_time}_frame_leads'
        file_names = os.listdir(dir_path)

        for file_name in file_names:
            if not file_name.endswith('.csv'):
                continue

            df = pd.read_csv(os.path.join(dir_path, file_name))
            method = file_name.split('.')[0]
            for metric in metrics:
                method_dict[method][metric].append(df[metric].mean())

        method_dict['data'].append(f'{data_name}_{lead_time}')

len(method_dict['data']), len(method_dict[method_lists[0]]['POD'])

Plot saved to output/forecast_comparison/images/philippines_13.269_123.832_3_frame_leads.png
Plot saved to output/forecast_comparison/images/philippines_13.269_123.832_5_frame_leads.png
Plot saved to output/forecast_comparison/images/KGRR_3_frame_leads.png
Plot saved to output/forecast_comparison/images/KGRR_5_frame_leads.png
Plot saved to output/forecast_comparison/images/KARX_3_frame_leads.png
Plot saved to output/forecast_comparison/images/KARX_5_frame_leads.png
Plot saved to output/forecast_comparison/images/KDVN_3_frame_leads.png
Plot saved to output/forecast_comparison/images/KDVN_5_frame_leads.png


(8, 8)

In [24]:
for metric in metrics:
    df = pd.DataFrame({'data': method_dict['data']})
    for method in method_lists:
        df[method] = method_dict[method][metric]
    
    print(f'=== {metric} ===')
    display(df)
    stored_path = f'output/forecast_comparison/summary/{metric}.csv'
    os.makedirs(os.path.dirname(stored_path), exist_ok=True)
    df.to_csv(stored_path, index=False)
    print('Store file successfully to', stored_path)

=== POD ===


,data,TITAN,ATA,ETITAN,ISCIT,our,STITAN
0,philippines_13.269_123.832_3,0.464928,0.539139,0.482965,0.354834,0.509241,0.481855
1,philippines_13.269_123.832_5,0.398065,0.450824,0.406916,0.251199,0.420962,0.422502
2,KGRR_3,0.313652,0.250515,0.306645,0.237833,0.416231,0.243848
3,KGRR_5,0.233877,0.166187,0.206348,0.155180,0.292898,0.150553
4,KARX_3,0.354244,0.288056,0.343029,0.288905,0.470976,0.276801
5,KARX_5,0.260519,0.187470,0.248591,0.206361,0.354183,0.186466
6,KDVN_3,0.396724,0.328349,0.307037,0.281654,0.441803,0.253946
7,KDVN_5,0.295527,0.219130,0.205094,0.177985,0.327115,0.165843


Store file successfully to output/forecast_comparison/summary/POD.csv
=== FAR ===


,data,TITAN,ATA,ETITAN,ISCIT,our,STITAN
0,philippines_13.269_123.832_3,0.513673,0.404285,0.483440,0.615809,0.464301,0.493270
1,philippines_13.269_123.832_5,0.559527,0.463241,0.532701,0.694127,0.528092,0.533456
2,KGRR_3,0.741748,0.780385,0.741644,0.797815,0.650625,0.790675
3,KGRR_5,0.827223,0.867653,0.842823,0.881318,0.775819,0.882886
4,KARX_3,0.637680,0.685579,0.629011,0.693214,0.504828,0.700062
5,KARX_5,0.726674,0.790529,0.720758,0.777567,0.616148,0.791258
6,KDVN_3,0.585523,0.652804,0.671927,0.700650,0.524161,0.726419
7,KDVN_5,0.685043,0.761558,0.778849,0.808746,0.638442,0.818209


Store file successfully to output/forecast_comparison/summary/FAR.csv
=== CSI ===


,data,TITAN,ATA,ETITAN,ISCIT,our,STITAN
0,philippines_13.269_123.832_3,0.323172,0.397191,0.333477,0.227633,0.352024,0.328493
1,philippines_13.269_123.832_5,0.271463,0.326137,0.275458,0.159007,0.281311,0.281723
2,KGRR_3,0.164770,0.132635,0.162871,0.122692,0.234422,0.127667
3,KGRR_5,0.110373,0.079464,0.097891,0.072052,0.145743,0.070888
4,KARX_3,0.217455,0.177206,0.218641,0.173476,0.316055,0.169185
5,KARX_5,0.152122,0.108934,0.150932,0.117765,0.221550,0.108853
6,KDVN_3,0.259425,0.205106,0.191041,0.171457,0.296981,0.155511
7,KDVN_5,0.183450,0.129806,0.120694,0.103164,0.206856,0.096752


Store file successfully to output/forecast_comparison/summary/CSI.csv
